In [1]:
import pandas as pd 
from numpy import int16

In [2]:
#DODAC ROZDIZAŁ NA ZLECENI Z PŁYTĄ PROSTOKĄTNĄ 1250x2500MM

In [3]:
#@title Funkcje
def generuj_tableke_na_cnc(_zlecenie):
  # zlecenie_ngl = _zlecenie.split("\n")[1].split("\t")[1:]
  zlecenie_ngl = ["MATERIAŁ",	"NUMER",	"ILOŚĆ",	"WYMIAR",	"OZNACZ.",	"CZAS"]
  zlecenie = list()
  for r in _zlecenie.split("\n")[2:]:
    r = r.split("\t")
    if len(r) == 7 and r[0] not in ["", "L.P."]:
      zlecenie.append(r[1:])

  df = pd.DataFrame(zlecenie, columns=zlecenie_ngl)

  df["GRUBOSC"] = df["MATERIAŁ"].apply(lambda x: x[-4:])


  def mat_typ(x):
    if "Sklejka" in x:
      return "SKL"
    elif "wiórowa" in x:
      return "PWI"

  def szer(x):
    new_str = ""
    x = x.lower()
    for c in x:
      if c not in "* ":
        new_str += c
    return new_str.strip().split("x")[0]

  def dlug(x):
    new_str = ""
    x = x.lower()
    for c in x:
      if c != "*" or c != " ":
        new_str += c
    return new_str.strip().split("x")[1]

  df["SZER"] = df["WYMIAR"].apply(szer).astype(int16)
  df["DLUG"] = df["WYMIAR"].apply(dlug).astype(int16)
  df["MAT_TYP"] = df["MATERIAŁ"].apply(mat_typ)
  df["ILOSC"] = df["ILOŚĆ"].astype(int16)

  return df

def drukuj_dane_cnc(df,MAT):

  _naglowek = "index$,material$,krok,sloje,grubosc,ilosc,mosty,nazwafmc$,kolor,fmcinclude$\n"
  content = list()
  content.append(_naglowek)
  # print(pwi_naglowek)
  for i in df[["NUMER", "ILOSC"]].values.tolist():
    if MAT == "S":
      a = [f"{i[0]}_SKL_X1_Msklejka,Sklejka,90,0,0,{i[1]},0,SKL",f"{i[0]}_SKL_X1_MSklejka.fmc,2,SKL","kontury"]
    elif MAT == "P":
      a = [f"{i[0]}_PWI_X1_MWIOR,WIOR,90,0,0,{i[1]},0,WIOR",f"{i[0]}_PWI_X1_MWIOR.fmc,2,WIOR","kontury"]
    b = chr(92).join(a)+chr(92)+"\n"
    # b = "/".join(a)+"/"+"\n"
    # print(b)
    content.append(b)

  content[-1] = content[-1].replace("\n", "")
  return content

def podziel_zlecenie(zlec):
  naglowki_zlecen = list(zlec.keys())

  for n in naglowki_zlecen:
    df = generuj_tableke_na_cnc(zlec[n])
    filtry = list(df.groupby(["MAT_TYP", "GRUBOSC"]).indices.keys())

    for f in filtry:
      if f[0] == "PWI":
        content = drukuj_dane_cnc(df[(df.MAT_TYP==f[0]) & (df.GRUBOSC == f[1])], "P")

      elif f[0] == "SKL":
        content = drukuj_dane_cnc(df[(df.MAT_TYP==f[0]) & (df.GRUBOSC == f[1]) & (df.DLUG <= 1525)], "S")

      fi = open(f"{n}_{f[0]}_{f[1].upper()}.csv", "w")
      # fi = open(f"{n}_{f[0]}.csv", "w")
      fi.writelines(content)
      fi.close()

def podziel_zlecenie_SKLP(zlec):
  naglowki_zlecen = list(zlec.keys())

  for n in naglowki_zlecen:
    df = generuj_tableke_na_cnc(zlec[n])
    filtry = list(df[(df.MAT_TYP=="SKL")].groupby(["MAT_TYP", "GRUBOSC"]).indices.keys())

    for f in filtry:

      content = drukuj_dane_cnc(df[(df.MAT_TYP=="SKL") & (df.GRUBOSC == f[1]) & (df.DLUG > 1525)], "S")

      fi = open(f"{n}_SKLP_{f[1].upper()}.csv", "w")
      # fi = open(f"{n}_{f[0]}.csv", "w")
      fi.writelines(content)
      fi.close()

In [14]:

#wzór na zlecenie
# """
# L.P.	MATERIAŁ	NUMER	ILOŚĆ	WYMIAR	OZNACZ.	CZAS
# 1	Płyta wiórowa 16mm	1278	40	*** 115x245	CN	20
# 2	Płyta wiórowa 16mm	1279	80	*** 110x222	CN	40
# 3	Płyta wiórowa 16mm	1393	40	*** 158x260	CN	20
_zlecenie = {

"AMA_4_1_24":
              """
L.P.	MATERIAŁ	NUMER	ILOŚĆ	WYMIAR	OZNACZ.	CZAS
1	Płyta wiórowa 16mm	1765	1155	*** 120x472	CN	4061:45 (211)
	RAZEM :		1155			4061:45



L.P.	MATERIAŁ	NUMER	ILOŚĆ	WYMIAR	OZNACZ.	CZAS
1	Sklejka 18mm	3003	260	*** 120x472	CN	914:20 (211)
2	Sklejka 18mm	3004	40	*** 140x757	CN	140:40 (211)
3	Sklejka 18mm	3005	40	*** 407x1070	CN	140:40 (211)
4	Sklejka 18mm	3006	40	*** 140x987	CN	140:40 (211)
5	Sklejka 18mm	3007	40	*** 450x1401	CN	140:40 (211)
6	Sklejka 18mm	3009	20	*** 397x1380	CN	70:20 (211)
7	Sklejka 18mm	3011	20	*** 411x1416	CN	70:20 (211)
8	Sklejka 18mm	3013	20	*** 457x1458	CN	70:20 (211)
9	Sklejka 18mm	3015	40	*** 415x1440	CN	140:40 (211)
10	Sklejka 18mm	3016	80	*** 357x1234	CN	281:20 (211)
11	Sklejka 18mm	3017	80	*** 140x765	CN	281:20 (211)
12	Sklejka 18mm	3018	40	*** 234x1635	CN	140:40 (211)
13	Sklejka 18mm	3019	40	*** 462X1216	CN	140:40 (211)
14	Sklejka 18mm	3020	40	*** 252x1653	CN	140:40 (211)
15	Sklejka 18mm	3021	40	*** 462x1234	CN	140:40 (211)
16	Sklejka 18mm	3022	40	*** 294x1677	CN	140:40 (211)
17	Sklejka 18mm	3023	40	*** 512x1300	CN	140:40 (211)
18	Sklejka 18mm	3024	80	*** 238x1660	CN	281:20 (211)
19	Sklejka 18mm	3025	80	*** 510x1280	CN	281:20 (211)
20	Sklejka 18mm	3026	40	*** 240x860	CN	140:40 (211)
21	Sklejka 18mm	3027	80	*** 240x842	CN	281:20 (211)
22	Sklejka 18mm	3029	40	*** 180x800	CN	140:40 (211)
23	Sklejka 18mm	3030	40	*** 180x782	CN	140:40 (211)
24	Sklejka 18mm	3031	30	*** 140x1100	CN	105:30 (211)
25	Sklejka 18mm	3034	20	*** 420x1080	CN	70:20 (211)
26	Sklejka 18mm	3035	10	*** 420x1260	CN	35:10 (211)
27	Sklejka 18mm	3036	10	*** 360x1200	CN	35:10 (211)
28	Sklejka 18mm	3037	10	*** 360x1182	CN	35:10 (211)
29	Sklejka 18mm	3038	20	*** 300x1310	CN	70:20 (211)
30	Sklejka 18mm	3039	455	*** 114x180	CN	1600:05 (211)
31	Sklejka 18mm	3041	65	*** 80x822	CN	228:35 (211)
32	Sklejka 18mm	3042	130	*** 80x822	CN	457:10 (211)
33	Sklejka 18mm	3043	15	*** 80x802	CN	52:45 (211)
34	Sklejka 18mm	3044	130	*** 114x632	CN	457:10 (211)
35	Sklejka 18mm	3045	470	*** 44x144	CN	1652:50 (211)
36	Sklejka 18mm	3046	10	*** 114x852	CN	35:10 (211)
37	Sklejka 18mm	3047	10	*** 154x1006	CN	35:10 (211)
38	Sklejka 18mm	3048	80	*** 114x558	CN	281:20 (211)
39	Sklejka 18mm	3049	40	*** 80x1225	CN	140:40 (211)
40	Sklejka 18mm	3050	40	*** 80x1205	CN	140:40 (211)
41	Sklejka 18mm	3051	40	*** 80x1075	CN	140:40 (211)
42	Sklejka 18mm	3052	20	*** 80x840	CN	70:20 (211)
43	Sklejka 18mm	3053	20	*** 80x860	CN	70:20 (211)
44	Sklejka 18mm	3054	20	*** 80x880	CN	70:20 (211)
45	Sklejka 18mm	3055	80	*** 80x365	CN	281:20 (211)
46	Sklejka 18mm	3056	40	*** 80x840	CN	140:40 (211)
47	Sklejka 18mm	3057	40	*** 80x878	CN	140:40 (211)
48	Sklejka 18mm	3058	10	*** 80x1042	CN	35:10 (211)
49	Sklejka 18mm	3059	10	*** 80x1202	CN	35:10 (211)
50	Sklejka 18mm	3060	10	*** 80x1182	CN	35:10 (211)
51	Sklejka 18mm	3061	10	*** 80x922	CN	35:10 (211)
52	Sklejka 18mm	3062	20	*** 80x700	CN	70:20 (211)
53	Sklejka 18mm	3063	10	*** 80x1202	CN	35:10 (211)
54	Sklejka 18mm	3064	115	*** 80x702	CN	404:25 (211)
55	Sklejka 18mm	3075	20	*** 140x1800	CN	70:20 (211)
56	Sklejka 18mm	3076	20	*** 140x1600	CN	70:20 (211)
57	Sklejka 18mm	3077	20	*** 140x1400	CN	70:20 (211)
58	Sklejka 18mm	3087	40	*** 140x900	CN	140:40 (211)
59	Sklejka 18mm	3108	5	*** 240x930	CN	17:35 (211)
60	Sklejka 18mm	3109	10	*** 240x912	CN	35:10 (211)
61	Sklejka 18mm	3110	5	*** 180x870	CN	17:35 (211)
62	Sklejka 18mm	3111	5	*** 180x852	CN	17:35 (211)
63	Sklejka 18mm	3125	60	*** 80x800	CN	211:00 (211)
64	Sklejka 18mm	3130	80	*** 140x1542	CN	281:20 (211)
65	Sklejka 18mm	3131	80	*** 188x889	CN	281:20 (211)
66	Sklejka 18mm	3133	30	*** 114x944	CN	105:30 (211)
67	Sklejka 18mm	3135	100	*** 125x125	CNC	351:40 (211)
	RAZEM :		3745			13169:55

 """

}


In [15]:
podziel_zlecenie(_zlecenie)

In [16]:
podziel_zlecenie_SKLP(_zlecenie)